### Mlxtend - 설치 필요

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import warnings
import random
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

import pymysql

# RDS 정보
rds_host = "db.diligentp.com"
rds_user = "park"
rds_password = "qkrwjdgus"
rds_db = "Oheumwan"

conn = pymysql.connect(host=rds_host, user=rds_user, passwd=rds_password, db=rds_db, connect_timeout=5)
sql = "SELECT * FROM TEST_AI_HISTORY"

def rule_base_model(find_name) :

    warnings.filterwarnings(action='ignore')
    %matplotlib inline
    
    # 데이터베이스에 있는 테이블 데이터 불러오기
    movie = pd.read_sql(sql, conn)
    # url = recipe_paths_url
    # movie = pd.read_csv(url + "TEST_AI_HISTORY.csv" , encoding = 'cp949')
    
    # 컬럼명 재설정
    movie.columns = ['member_id' , 'CKG_NM', 'used_ingredients']
    
    # 추천 알고리즘 분석에 쓰이는 컬럼(사용자 ID , recipe name)으로 데이터 프레임 추출
    movie['member_id'] = movie['member_id'].astype(str)
    movie = movie[movie['CKG_NM'].notnull()].reset_index(drop=True)
    agg = movie.groupby(['member_id'])['CKG_NM'].agg({'unique'})
    
    # 학습데이터 준비(fit & transform)
    te = TransactionEncoder()
    te_ary = te.fit(agg['unique'].values).transform(agg['unique'].values)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    # apriori 분석 알고리즘으로 평가지표 추출(데이터 프레임)
    frequent_itemsets = apriori(df, min_support=0.25, use_colnames=True)
    eval_df = association_rules(frequent_itemsets , metric="confidence", min_threshold= 0.25)
    display(eval_df)
    # 지표를 바탕으로 요리명 추출
    find_df = eval_df[eval_df['antecedents'].astype(str).str.lower().str.contains(find_name)]
    display(find_df)
    find_df['consequents']=  find_df['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")       
    print(find_df['consequents'])
    # 요리명 추천(리스트 형)
    recommend_lst = list(set(find_df['consequents']))
    result_null = list(set(movie[movie['CKG_NM'] != find_name]['CKG_NM']))
    len_num = len(recommend_lst)

    # 결과 리스트(중복 X, 랜덤 O) - 변경 사항 : for ~ if ~ else, lst_num == 0 일때 , 전체 히스토리 데이터셋에서 랜덤으로 2개 추출
    for i in range(10 , 0 , -1) :
        if len_num >= i:
            return random.sample(recommend_lst , i)
    else :
        return random.sample(result_null , 2) # 전체 히스토리 데이터에서 랜덤으로 2개만 뽑아오기

    # 결과 리스트(중복 X, 랜덤 O) - 변경 사항 : if ~ else, lst_num == 0 일때 ,'추천메뉴가 없어요'추출
    
    #     lst_null = ["추천메뉴가 없어요"] 

    #     if len_num >= 10 :
    #         return random.sample(recommend_lst , 10)
    #     elif len_num >= 9 :
    #         return random.sample(recommend_lst , 9)
    #     elif len_num >= 8 :
    #         return random.sample(recommend_lst , 8)
    #     elif len_num >= 7 :
    #         return random.sample(recommend_lst , 7)
    #     elif len_num >= 6 :
    #         return random.sample(recommend_lst , 6)
    #     elif len_num >= 5 :
    #         return random.sample(recommend_lst , 5)
    #     elif len_num >= 4 :
    #         return random.sample(recommend_lst , 4)
    #     elif len_num >= 3 :
    #         return random.sample(recommend_lst , 3)
    #     elif len_num >= 2 :
    #         return random.sample(recommend_lst , 2)
    #     elif len_num ==  0:
    #         return random.sample(lst_null , 2) # 전체 히스토리 데이터에서 랜덤으로 2개만 뽑아오기
    #     else :
    #         return random.sample(recommend_lst , 1)

In [4]:
rule_base_model('고추튀김')  # 사용자가 요리한 음식 


# GPT
# 재료 -> 레시피 -> 히스토리 

# AI 코드
# 메뉴에 오늘의 추천 요리를 만들어서 
# 가장 최근에 사용자의 히스토리 요리 이력의 요리명 -> 연관된 레시피 추천

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(고추튀김),((아기반찬) 동그랑땡),0.50,0.25,0.25,0.5,2.0,0.1250,1.5,1.000000
1,((아기반찬) 동그랑땡),(고추튀김),0.25,0.50,0.25,1.0,2.0,0.1250,inf,0.666667
2,(깻잎지),(가쓰오우동),0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000
3,(가쓰오우동),(깻잎지),0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000
4,(갈릭치킨),(쫄면),0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000
...,...,...,...,...,...,...,...,...,...,...
99,"(연어타다끼, 영양 가득 비빔밥 만들기)",(참소스),0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000
100,"(참소스, 영양 가득 비빔밥 만들기)",(연어타다끼),0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000
101,(연어타다끼),"(참소스, 영양 가득 비빔밥 만들기)",0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000
102,(참소스),"(연어타다끼, 영양 가득 비빔밥 만들기)",0.25,0.25,0.25,1.0,4.0,0.1875,inf,1.000000


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(고추튀김),((아기반찬) 동그랑땡),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
7,(고추튀김),(감자짜글이),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
8,(고추튀김),(고구마그라탕),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
10,(고추튀김),(꽃게볶음밥),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
12,(고추튀김),(미니새우버거),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
14,(고추튀김),(애호박볶음),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
16,(고추튀김),(오리엔탈소스),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
18,(고추튀김),(오징어순대),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
21,(고추튀김),(오징어순대 만드는 법 담백하고 깔끔한 맛에 하트 뿅!),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0
22,(고추튀김),(충무김밥),0.50,0.25,0.25,0.5,2.000000,0.1250,1.500,1.0


0                               3 : (아기반찬) 동그랑땡
7                                     3 : 감자짜글이
8                                    3 : 고구마그라탕
10                                    3 : 꽃게볶음밥
12                                   3 : 미니새우버거
14                                    3 : 애호박볶음
16                                   3 : 오리엔탈소스
18                                    3 : 오징어순대
21            3 : 오징어순대 만드는 법 담백하고 깔끔한 맛에 하트 뿅!
22                                     3 : 충무김밥
24                                 3 : 카레새우삼각김밥
26                                    3 : 표고버섯전
28                                    3 : 현미호두죽
74                                    3 : 오징어순대
75                                    3 : 애호박볶음
77                             3 : 애호박볶음, 오징어순대
81                                   3 : 오리엔탈소스
82            3 : 오징어순대 만드는 법 담백하고 깔끔한 맛에 하트 뿅!
85    3 : 오징어순대 만드는 법 담백하고 깔끔한 맛에 하트 뿅!, 오리엔탈소스
Name: consequents, dtype: object


['꽃게볶음밥',
 '오징어순대 만드는 법 담백하고 깔끔한 맛에 하트 뿅!',
 '애호박볶음, 오징어순대',
 '고구마그라탕',
 '충무김밥',
 '현미호두죽',
 '표고버섯전',
 '애호박볶음',
 '미니새우버거',
 '오리엔탈소스']